In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
# Librerias de uso común
import numpy as np
import torch
from scipy.spatial import distance
import os, json, cv2, random, shutil
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow

# Utilidades del detectron2

from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2 import structures
from detectron2.structures.boxes import Boxes
from detectron2 import model_zoo
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper, build_detection_train_loader   # the default mapper

In [3]:
register_coco_instances("dataset_train", {}, "Dataset/train.json", "Dataset/data")
register_coco_instances("dataset_test", {}, "Dataset/test.json", "Dataset/data")

In [4]:
MetadataCatalog.get("dataset_test")

namespace(name='dataset_test',
          json_file='Dataset/test.json',
          image_root='Dataset/data',
          evaluator_type='coco')

In [5]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dataset_train",)
cfg.DATASETS.TEST = ("dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 25000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.TEST.DTECTIONS_PER_IMAGE = 20

In [6]:
dataloader = build_detection_train_loader(cfg,
   mapper=DatasetMapper(cfg, is_train=True, augmentations=[
      T.Resize((391, 417))
   ]))

[01/06 16:16:51 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [Resize(shape=(391, 417))]
WARNING [01/06 16:16:51 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/06 16:16:51 d2.data.datasets.coco]: Loaded 22 images in COCO format from Dataset/train.json
[01/06 16:16:51 d2.data.build]: Removed 0 images with no usable annotations. 22 images left.
[01/06 16:16:51 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   diente   | 12           |    raiz    | 11           |
|            |              |            |              |
|   total    | 23           |            |              |
[01/06 16:16:51 d2.data.build]: Using training sampler TrainingSampler
[01/06 16:16:51 d2.data.common]: Serializing 22 elements to byte tensors and concatenating them all ...
[01/06 16:

In [7]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

[01/06 16:16:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/home/asl1014/miniconda3/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/asl1014/miniconda3/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/06 16:16:58 d2.utils.events]:  eta: 0:32:48  iter: 15019  total_loss: 0.03302  loss_cls: 0.00463  loss_box_reg: 0.009962  loss_mask: 0.01605  loss_rpn_cls: 1.256e-05  loss_rpn_loc: 0.0009691  time: 0.1982  data_time: 0.0122  lr: 0.00025  max_mem: 2338M
[01/06 16:17:02 d2.utils.events]:  eta: 0:32:27  iter: 15039  total_loss: 0.03458  loss_cls: 0.005107  loss_box_reg: 0.01148  loss_mask: 0.01671  loss_rpn_cls: 6.761e-06  loss_rpn_loc: 0.0009272  time: 0.1931  data_time: 0.0048  lr: 0.00025  max_mem: 2338M
[01/06 16:17:06 d2.utils.events]:  eta: 0:32:24  iter: 15059  total_loss: 0.03305  loss_cls: 0.00534  loss_box_reg: 0.01016  loss_mask: 0.01626  loss_rpn_cls: 4.968e-06  loss_rpn_loc: 0.0009658  time: 0.1914  data_time: 0.0040  lr: 0.00025  max_mem: 2338M
[01/06 16:17:10 d2.utils.events]:  eta: 0:32:18  iter: 15079  total_loss: 0.03384  loss_cls: 0.004671  loss_box_reg: 0.01004  loss_mask: 0.01689  loss_rpn_cls: 1.763e-05  loss_rpn_loc: 0.0009603  time: 0.1916  data_time: 0.0039  l